In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

In [3]:
data = pd.read_csv("Task 3 and 4_Loan_Data.csv")

In [4]:
data.head()

,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0


In [5]:
# extract FICO scores from the data
fico_scores = data['fico_score']

# define the number of buckets
num_buckets = 5

# we're going with the mean squared error approach
optimization_approach = 'mse'  

In [7]:
fico_scores.head()

0    605
1    572
2    602
3    612
4    631
Name: fico_score, dtype: int64

In [8]:
def quantize_fico_scores(fico_scores, num_buckets, optimization_approach):
    if optimization_approach == 'mse':
        # mean squared error optimization
        fico_scores_reshaped = fico_scores.values.reshape(-1, 1)
        kmeans = KMeans(n_clusters=num_buckets, random_state=0).fit(fico_scores_reshaped)
        boundaries = np.sort(kmeans.cluster_centers_.flatten())
    elif optimization_approach == 'log_likelihood':
        pass  

    return boundaries

In [9]:
# make boundaries
optimal_boundaries = quantize_fico_scores(fico_scores, num_buckets, optimization_approach)

In [10]:
# map out FICO scores to ratings based on the buckets we made before
def map_fico_to_ratings(fico_scores, boundaries):
    ratings = np.searchsorted(boundaries, fico_scores, side='right')
    # lower rating signifies a better credit score
    ratings = num_buckets - ratings + 1  
    return ratings

In [11]:
# generate rating map
rating_map = {fico: rating for fico, rating in zip(optimal_boundaries, range(1, num_buckets+1))}

In [12]:
print("Rating Map:")
for boundary, rating in rating_map.items():
    print(f"FICO Score <= {boundary}: Rating {rating}")

# update the 'default' column based on the rating map
data['rating'] = map_fico_to_ratings(data['fico_score'], optimal_boundaries)

# save the updated data to a new CSV file
data.to_csv('loan_data_with_ratings.csv', index=False)

Rating Map:
FICO Score <= 522.0664993726475: Rating 1
FICO Score <= 583.435267857143: Rating 2
FICO Score <= 631.9098546042004: Rating 3
FICO Score <= 678.7661444401989: Rating 4
FICO Score <= 735.8137490007996: Rating 5
